In [75]:
import io
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\milam\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [79]:
training_data, testing_data = imdb['train'], imdb['test']

In [191]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [192]:
for s,l in training_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())
for s,l in testing_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [172]:
num_words = 10000
oov_token = "<OOV>"
maxlen = 120
truncating = "post"
embedding_dim = 16

In [193]:
tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=maxlen, truncating=truncating)

In [194]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=maxlen)

In [195]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [196]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
flatten_13 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 6)                 11526     
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [197]:
epochs = 10
model.fit(padded, training_labels, epochs=epochs, validation_data = (testing_padded, testing_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 191us/sample - loss: 0.4924 - accuracy: 0.7432 - val_loss: 0.3392 - val_accuracy: 0.8506
Epoch 2/10
25000/25000 [==============================] - 4s 167us/sample - loss: 0.2382 - accuracy: 0.9079 - val_loss: 0.3734 - val_accuracy: 0.8395
Epoch 3/10
25000/25000 [==============================] - 3s 115us/sample - loss: 0.0843 - accuracy: 0.9790 - val_loss: 0.4533 - val_accuracy: 0.8282
Epoch 4/10
25000/25000 [==============================] - 3s 114us/sample - loss: 0.0201 - accuracy: 0.9976 - val_loss: 0.5326 - val_accuracy: 0.8262
Epoch 5/10
25000/25000 [==============================] - 3s 121us/sample - loss: 0.0049 - accuracy: 0.9998 - val_loss: 0.5912 - val_accuracy: 0.8266
Epoch 6/10
25000/25000 [==============================] - 3s 118us/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6394 - val_accuracy: 0.8285
Epoch 7/10
25000/25000 [==========================

In [198]:
# embeddings layer is layer 0
e = model.layers[0]
weights = e.get_weights()[0] 
print(weights.shape) #shape: num_words, embedding_dim

(10000, 16)


In [199]:
reverse_word_index = dict([(val, word) for (word, val) in word_index.items()])

In [200]:
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, num_words):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    out_v.write("\t".join([str(x) for x in embeddings]) + "\n")
    
out_m.close()
out_v.close()
